In [ ]:
from google.colab import files
data = files.upload() #load the train.csv file

In [ ]:
import pandas as pd
import io
df = pd.read_csv(io.StringIO(data['train.csv'].decode('utf-8')))

In [ ]:
df_features = df.drop('target', axis=1)
df_labels = df['target']
df_features

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
dfSparse = enc.fit_transform(df_features)
dfSparse

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# encode the labels to numeric values
le = LabelEncoder()
df_labels = le.fit_transform(df_labels)
df_train_features, df_test_features, df_train_labels, df_test_labels = train_test_split(dfSparse, df_labels, test_size = 0.7)
df_train_features

In [ ]:
from xgboost import XGBClassifier
params = { "n_estimators": 300, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor' }
model = XGBClassifier(**params)
model

In [ ]:
model.fit(df_train_features, df_train_labels)

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [2, 4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.5],
    'min_child_weight': [1, 5, 10],
    'gamma': [0, 0.1, 0.5]
}

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
param_comb = 5
folds=5
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=param_comb,  n_jobs=-1, 
                                   cv=skf.split(df_train_features,df_train_labels), verbose=3)

In [ ]:
%%time 
grid_result = random_search.fit(df_train_features, df_train_labels)

In [ ]:
random_search.best_params_

In [ ]:
data1 = files.upload()
test_features = pd.read_csv(io.StringIO(data1['test.csv'].decode('utf-8')))

In [ ]:
predictions = random_search.best_estimator_.predict(test_features)
predictions = predictions[:43315]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(df_test_labels, predictions)

his code loads a CSV file containing data for training a machine learning model, preprocesses the data by encoding categorical features using One-Hot Encoding, splits the data into training and testing sets, fits an XGBoost classifier model to the training data, and performs hyperparameter tuning using RandomizedSearchCV to find the best combination of hyperparameters for the model. It then loads a separate CSV file containing the test data, uses the best model to predict the target variable for the test data, and computes the accuracy of the predictions against the true labels.